# Initialization and Data Setup

In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from scipy import stats
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import (
    roc_auc_score, accuracy_score, balanced_accuracy_score,
    confusion_matrix, classification_report
)


In [2]:
bailey = pd.read_excel("EP_DX_Bailey_11_06_2024_2.xlsx")
ibis = pd.read_csv("vineland_filtered.csv")
eacsf = pd.read_csv("EACSF_SumData_AAL_09_2025_mm3_corrected.csv")
tissue = pd.read_csv("EPinfDS_V06_TissueSegVols_v1.1_2025_08_07_with_numeric.csv")

In [3]:
ibis['CandID'] = ibis['CandID'].astype(str).str.strip()
eacsf['Case'] = eacsf['Case'].astype(str).str.strip()

In [4]:
merged = pd.merge(eacsf, ibis, left_on='Case', right_on='CandID', how='inner')

print(f"Merged dataset shape: {merged.shape}")
merged["CandID"] = merged["CandID"].astype(str).str.strip()
tissue["CandID_numeric"] = tissue["CandID_numeric"].astype(str).str.strip()

merged2 = pd.merge(
    merged,
    tissue,
    how="left",
    left_on="Identifiers",
    right_on="CandID_numeric"
)

#### edited in Excel, loading datafile

merged2 = pd.read_csv("merged2.csv")

Merged dataset shape: (71, 226)


# Creating Group Datasets

In [5]:
ds_infant = merged2.loc[merged2['Cohort'] == 'DS Infant']
ds_control = merged2.loc[merged2['Cohort'] == 'Control DS Infant']


# Normalization Function

In [6]:
def normalized_eacsf(df):
    df = df.copy()

    """
    if 'vol05_mm^3_ICV' not in df.columns:
        # Prepare lookup from the tissue vols dataframe
        icv_lookup = icv_source_df[['CandID_numeric', 'vol05_mm^3_ICV']].drop_duplicates()
        
        # Merge on Case (EACSF ID) == CandID_numeric (ICV ID)
        df_out = pd.merge(
            df_out, 
            icv_lookup, 
            left_on='Case', 
            right_on='CandID_numeric', 
            how='left'
        )
    """    
    # loop through AAL regions and add a normalized value
    roi_cols = [
    1, 3, 5, 7, 9, 11, 13, 15, 17, 19,
    21, 23, 25, 27, 29, 31, 33, 35, 39, 43,
    45, 47, 49, 51, 53, 55, 57, 59, 61, 63,
    65, 67, 69, 79, 81, 83, 85, 87, 89,
    2, 4, 6, 8, 10, 12, 14, 16, 18, 20,
    22, 24, 26, 28, 30, 32, 34, 36, 40, 44,
    46, 48, 50, 52, 54, 56, 58, 60, 62, 64,
    66, 68, 70, 80, 82, 84, 86, 88, 90]

    for col in roi_cols:
        norm_col_name = f"{str(col)}_norm_ICV"
        df[norm_col_name] = df[str(col)] / df['vol05_mm^3_ICV']

    return df

# Normalizing DS Datasets

In [7]:
# normalizing all of the groups
ds_infant_norm = normalized_eacsf(ds_infant)
ds_control_norm = normalized_eacsf(ds_control)

In [8]:
print("\nSample Data (ds_infant_norm):")
cols_to_show = ['Case', 'Total_EACSF', 'vol05_mm^3_ICV', '1_norm_ICV']
# Filter cols_to_show to only those present
cols_to_show = [c for c in cols_to_show if c in ds_infant_norm.columns]
print(ds_infant_norm[cols_to_show].head())



Sample Data (ds_infant_norm):
      Case  vol05_mm^3_ICV  1_norm_ICV
4   151582        765448.0    0.006789
5   159592        748576.0    0.009237
8   244858        643820.0    0.008790
13  328191        819326.0    0.011830
14  336515        656811.2    0.005601


In [9]:
ds_infant_norm.to_csv("ds_infant_normalized_eacsf.csv", index=False)
ds_control_norm.to_csv("ds_control_normalized_eacsf.csv", index=False)

# Running T-Test on Normalized DS Infants vs. Controls

In [10]:
# t-test

aal_cols = [
    c for c in ds_infant_norm.columns
    if c.endswith("_norm_ICV")]

print(f"Found {len(aal_cols)} AAL regions")

results = []

for col in aal_cols:
    ds_vals = ds_infant_norm[col].dropna()
    ctrl_vals = ds_control_norm[col].dropna()

    t_stat, p_val = ttest_ind(
        ds_vals,
        ctrl_vals,
        equal_var=False,
        nan_policy="omit"
    )

    results.append({
        "AAL_region": col.replace("_norm_ICV", ""),
        "column_name": col,
        "n_DS": len(ds_vals),
        "n_Control": len(ctrl_vals),
        "mean_DS": ds_vals.mean(),
        "mean_Control": ctrl_vals.mean(),
        "t_stat": t_stat,
        "p_value": p_val
    })

# Convert to DataFrame
ttest_results_df = pd.DataFrame(results)

#remove junk columns
ttest_results_df = ttest_results_df[ttest_results_df['AAL_region'] != '0']
ttest_results_df = ttest_results_df[ttest_results_df['AAL_region'] != '0.1']

print(ttest_results_df.head())

ttest_results_df.to_csv("ds_ttest_results.csv", index=False)

Found 78 AAL regions
  AAL_region column_name  n_DS  n_Control   mean_DS  mean_Control    t_stat  \
0          1  1_norm_ICV    36         35  0.007319      0.006949  0.773979   
1          3  3_norm_ICV    36         35  0.008568      0.009697 -1.488405   
2          5  5_norm_ICV    36         35  0.002815      0.002813  0.019767   
3          7  7_norm_ICV    36         35  0.011181      0.011530 -0.406024   
4          9  9_norm_ICV    36         35  0.001799      0.001758  0.370046   

    p_value  
0  0.441723  
1  0.141216  
2  0.984287  
3  0.685982  
4  0.712503  


# T-tests for Covariates – Maternal Age, Sex Ratios, and Age at Scan

### Maternal Age

In [11]:
pd.set_option('future.no_silent_downcasting', True)
mage_col_cvd = "V06-CVD demographics,Maternal_age_at_birth"
mage_col_noncvd = "V06 demographics,Maternal_age_at_birth"

def mage(df, col1, col2):
    s1 = pd.to_numeric(df[col1].replace(".", np.nan), errors="coerce")
    s2 = pd.to_numeric(df[col2].replace(".", np.nan), errors="coerce")
    return s1.combine_first(s2)   # take s1 if present, else s2

results = []

ds_mage_vals = mage(ds_infant_norm, mage_col_cvd, mage_col_noncvd).dropna()
ctrl_mage_vals = mage(ds_control_norm, mage_col_cvd, mage_col_noncvd).dropna()

t_stat, p_val = ttest_ind(
    ds_mage_vals,
    ctrl_mage_vals,
    equal_var=False,
    nan_policy="omit"
)

results.append({
    "column_name": 'Mat Age at Birth',
    "n_DS": len(ds_mage_vals),
    "n_Control": len(ctrl_mage_vals),
    "mean_DS": ds_mage_vals.mean(),
    "mean_Control": ctrl_mage_vals.mean(),
    "t_stat": t_stat,
    "p_value": p_val
    })

# Convert to DataFrame
mage_ttest_results_df = pd.DataFrame(results)
print(mage_ttest_results_df.head())

        column_name  n_DS  n_Control  mean_DS  mean_Control   t_stat   p_value
0  Mat Age at Birth    35         35    431.6    406.142857  1.92932  0.058396


#### Low p-value indicates potential bias in maternal age at birth. Control group is younger on average than the DS infant group

### Age at Scan

In [12]:
import warnings

warnings.filterwarnings(
    "ignore",
    message="Could not infer format, so each element will be parsed individually*",
    category=UserWarning
)

visit_date_col_cvd = "V06-CVD demographics,Appointment_MRI_date"
visit_date_col_noncvd = "V06 demographics,Appointment_MRI_date"

dob_col_cvd = "V06-CVD demographics,DoB"
dob_col_noncvd = "V06 demographics,DoB"

def age_at_scan(df, visit_cvd_col, visit_col, dob_cvd_col, dob_col):
    visit_dates_1 = pd.to_datetime(df[visit_cvd_col].replace("", np.nan), errors="coerce")
    visit_dates_2 = pd.to_datetime(df[visit_col].replace("", np.nan), errors="coerce")
    visit_dates = visit_dates_1.combine_first(visit_dates_2)

    dob_dates_1 = pd.to_datetime(df[dob_cvd_col].replace("", np.nan), errors="coerce")
    dob_dates_2 = pd.to_datetime(df[dob_col].replace("", np.nan), errors="coerce")
    dob_dates = dob_dates_1.combine_first(dob_dates_2)
        
    age_days = (visit_dates - dob_dates).dt.days
    age_years = age_days / 365.25
    return age_years

ds_age_vals = age_at_scan(ds_infant_norm, visit_date_col_cvd, visit_date_col_noncvd, dob_col_cvd, dob_col_noncvd).dropna()
ctrl_age_vals = age_at_scan(ds_control_norm, visit_date_col_cvd, visit_date_col_noncvd, dob_col_cvd, dob_col_noncvd).dropna()
t_stat, p_val = ttest_ind(
    ds_age_vals,
    ctrl_age_vals,
    equal_var=False,
    nan_policy="omit"
)
results = []
results.append({
    "column_name": 'Age at Scan',
    "n_DS": len(ds_age_vals),
    "n_Control": len(ctrl_age_vals),
    "mean_DS": ds_age_vals.mean(),
    "mean_Control": ctrl_age_vals.mean(),
    "mean_DS_days": ds_age_vals.mean() * 365.25,
    "mean_Control_days": ctrl_age_vals.mean() * 365.25,
    "t_stat": t_stat,
    "p_value": p_val
    })
# Convert to DataFrame
age_ttest_results_df = pd.DataFrame(results)
print(age_ttest_results_df.head())

   column_name  n_DS  n_Control   mean_DS  mean_Control  mean_DS_days  \
0  Age at Scan    36         35  0.565746      0.535054    206.638889   

   mean_Control_days    t_stat   p_value  
0         195.428571  2.106865  0.040244  


#### No significant difference in age at scan

### Sex

In [13]:
sex_col_cvd = "V06-CVD demographics,Sex"
sex_col_noncvd = "V06 demographics,Sex"

def sex(df, col1, col2):
    s1 = pd.Series(df[sex_col_cvd].replace(".", np.nan), index=df.index)
    s2 = pd.Series(df[sex_col_noncvd].replace(".", np.nan), index=df.index)
    sex = s1.combine_first(s2)

    # Normalize strings
    sex = sex.astype(str).str.strip().str.lower()

    # Map sex → numeric
    sex_map = {
        "female": 0,
        "male": 1
    }

    sex_numeric = sex.map(sex_map)

    return sex_numeric


ds_sex_vals = sex(ds_infant_norm, sex_col_cvd, sex_col_noncvd).dropna()
ctrl_sex_vals = sex(ds_control_norm, sex_col_cvd, sex_col_noncvd).dropna()

t_stat, p_val = ttest_ind(
    ds_sex_vals,
    ctrl_sex_vals,
    equal_var=False,
    nan_policy="omit"
)

sex_ttest_results_df = pd.DataFrame([{
    "column_name": "Sex (F=0, M=1)",
    "n_DS": len(ds_sex_vals),
    "n_Control": len(ctrl_sex_vals),
    "mean_DS": ds_sex_vals.mean(),
    "mean_Control": ctrl_sex_vals.mean(),
    "t_stat": t_stat,
    "p_value": p_val
}])

print(sex_ttest_results_df)

      column_name  n_DS  n_Control   mean_DS  mean_Control    t_stat   p_value
0  Sex (F=0, M=1)    36         35  0.444444      0.571429 -1.063474  0.291277


#### No significant difference in sex. DS infants have slightly more females, controls have slightly more males, but not significant enough with roughly 0.3 p-value

# Setting up ANOVA

In [14]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

aal_col = "17_norm_ICV"   # test region

# -----------------------
ds_df = pd.DataFrame({
    "y": pd.to_numeric(ds_infant_norm[aal_col], errors="coerce"),
    "Group": "DS",
    "Sex": sex(ds_infant_norm, sex_col_cvd, sex_col_noncvd),
    "MaternalAge": mage(ds_infant_norm, mage_col_cvd, mage_col_noncvd),
    "AgeAtScan": age_at_scan(ds_infant_norm, visit_date_col_cvd, visit_date_col_noncvd, dob_col_cvd, dob_col_noncvd),
})

ctrl_df = pd.DataFrame({
    "y": pd.to_numeric(ds_control_norm[aal_col], errors="coerce"),
    "Group": "Control",
    "Sex": sex(ds_control_norm, sex_col_cvd, sex_col_noncvd),
    "MaternalAge": mage(ds_control_norm, mage_col_cvd, mage_col_noncvd),
    "AgeAtScan": age_at_scan(ds_control_norm, visit_date_col_cvd, visit_date_col_noncvd, dob_col_cvd, dob_col_noncvd),
})

df_ancova = pd.concat([ds_df, ctrl_df], ignore_index=True)

# Drop rows missing any required variable
df_ancova = df_ancova.dropna(subset=["y", "Group", "Sex", "MaternalAge", "AgeAtScan"])

print(f"Using AAL column: {aal_col}")
print("N used (after dropping missing):", len(df_ancova))
print(df_ancova["Group"].value_counts())

# -----------------------
# Fit ANCOVA model
# -----------------------
# C(Group) makes Group categorical; DS vs Control effect is the main thing you care about.
model = smf.ols("y ~ C(Group) + Sex + MaternalAge + AgeAtScan", data=df_ancova).fit()

# Type II ANOVA table (common choice when you don't have complex interactions)
anova_table = anova_lm(model, typ=2)

print("\nANCOVA (Type II ANOVA) table:")
display(anova_table)

print("\nModel coefficients (for interpretation):")
display(model.summary2().tables[1])

Using AAL column: 17_norm_ICV
N used (after dropping missing): 70
Group
DS         35
Control    35
Name: count, dtype: int64

ANCOVA (Type II ANOVA) table:


,sum_sq,df,F,PR(>F)
C(Group),5.222563e-06,1.0,15.922981,0.000170
Sex,2.729802e-06,1.0,8.322844,0.005306
MaternalAge,4.149322e-10,1.0,0.001265,0.971736
AgeAtScan,1.883717e-07,1.0,0.574323,0.451285
Residual,2.131929e-05,65.0,NaN,NaN



Model coefficients (for interpretation):


,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,2.040635e-03,0.000784,2.603026,0.011435,0.000475,0.003606
C(Group)[T.DS],5.801384e-04,0.000145,3.990361,0.000170,0.000290,0.000870
Sex,4.017052e-04,0.000139,2.884934,0.005306,0.000124,0.000680
MaternalAge,-4.522679e-08,0.000001,-0.035568,0.971736,-0.000003,0.000002
AgeAtScan,-8.563509e-04,0.001130,-0.757841,0.451285,-0.003113,0.001400


## Looping ANOVA across all regions

In [15]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

# --- 1) Identify AAL region columns ---
ds_infant_norm.columns = ds_infant_norm.columns.map(str)
ds_control_norm.columns = ds_control_norm.columns.map(str)

ds_infant_norm = ds_infant_norm.drop(columns=[0, 0.1, "0_norm_ICV", "0.1_norm_ICV"], errors="ignore")
ds_control_norm = ds_control_norm.drop(columns=[0, 0.1, "0_norm_ICV", "0.1_norm_ICV"], errors="ignore")
aal_cols = [c for c in ds_infant_norm.columns if c.endswith("_norm_ICV")]

results = []

for aal_col in aal_cols:
    # --- 2) Build combined ANCOVA dataframe (reuse your exact covariate construction) ---
    ds_df = pd.DataFrame({
        "y": pd.to_numeric(ds_infant_norm[aal_col], errors="coerce"),
        "Group": "DS",
        "Sex": sex(ds_infant_norm, sex_col_cvd, sex_col_noncvd),
        "MaternalAge": mage(ds_infant_norm, mage_col_cvd, mage_col_noncvd),
        "AgeAtScan": age_at_scan(ds_infant_norm, visit_date_col_cvd, visit_date_col_noncvd,
                                       dob_col_cvd, dob_col_noncvd),
    })

    ctrl_df = pd.DataFrame({
        "y": pd.to_numeric(ds_control_norm[aal_col], errors="coerce"),
        "Group": "Control",
        "Sex": sex(ds_control_norm, sex_col_cvd, sex_col_noncvd),
        "MaternalAge": mage(ds_control_norm, mage_col_cvd, mage_col_noncvd),
        "AgeAtScan": age_at_scan(ds_control_norm, visit_date_col_cvd, visit_date_col_noncvd,
                                       dob_col_cvd, dob_col_noncvd),
    })

    df_ancova = pd.concat([ds_df, ctrl_df], ignore_index=True)
    df_ancova = df_ancova.dropna(subset=["y", "Group", "Sex", "MaternalAge", "AgeAtScan"])

    # Skip if too few samples to fit reliably
    if df_ancova["Group"].nunique() < 2 or len(df_ancova) < 10:
        continue

    # --- 3) Fit model + ANOVA table ---
    try:
        model = smf.ols("y ~ C(Group) + Sex + MaternalAge + AgeAtScan", data=df_ancova).fit()
        aov = anova_lm(model, typ=2)

        # --- 4) Extract PR(>F) for C(Group) ---
        p_group = aov.loc["C(Group)", "PR(>F)"] if "C(Group)" in aov.index else np.nan

        results.append({
            "AAL_region": int(aal_col.replace("_norm_ICV", "")),
            "column_name": aal_col,
            "n_used": len(df_ancova),
            "p_group": p_group
        })

    except Exception as e:
        # Store failure info (useful for debugging specific regions)
        results.append({
            "AAL_region": int(aal_col.replace("_norm_ICV", "")),
            "column_name": aal_col,
            "n_used": len(df_ancova),
            "p_group": np.nan,
            "error": str(e)
        })

# --- 5) Results dataframe ---
group_pvals_df = pd.DataFrame(results).sort_values("AAL_region").reset_index(drop=True)
group_pvals_df

,AAL_region,column_name,n_used,p_group
0,1,1_norm_ICV,70,0.234570
1,2,2_norm_ICV,70,0.750031
2,3,3_norm_ICV,70,0.252611
3,4,4_norm_ICV,70,0.023634
4,5,5_norm_ICV,70,0.807600
...,...,...,...,...
73,86,86_norm_ICV,70,0.000015
74,87,87_norm_ICV,70,0.054966
75,88,88_norm_ICV,70,0.109850
76,89,89_norm_ICV,70,0.097334


take p-values, put in text file, use meshmath (look at reference video from Teams --> lookupPointData)

create a csv that has AAL region in one column, and 2nd column has value that we want to replace with (p-value)

load into SPV

In [16]:
# Use your existing ANCOVA results
df = group_pvals_df.copy()
df["neglog10p"] = -np.log10(df["p_group"].clip(lower=1e-300))

# Save region → value mapping
df_out = df[["AAL_region", "neglog10p"]].sort_values("AAL_region")
df_out.to_csv("AAL_region_neglog10p.csv", index=False)

# Table Exports

In [17]:
from docx import Document
import pandas as pd
import numpy as np

def round_df(df, decimals=3):
    df_out = df.copy()
    num_cols = df_out.select_dtypes(include=[np.number]).columns
    df_out[num_cols] = df_out[num_cols].round(decimals)
    return df_out

def dataframe_to_word_table(doc, df, title=None):
    if title:
        doc.add_heading(title, level=2)

    table = doc.add_table(rows=1, cols=len(df.columns))
    table.style = "Table Grid"

    # Header row
    hdr_cells = table.rows[0].cells
    for i, col in enumerate(df.columns):
        hdr_cells[i].text = str(col)

    # Data rows
    for _, row in df.iterrows():
        row_cells = table.add_row().cells
        for i, val in enumerate(row):
            row_cells[i].text = str(val)

In [18]:
doc = Document()
doc.add_heading("Statistical Results", level=1)

dataframe_to_word_table(
    doc,
    round_df(age_ttest_results_df, 3),
    "Age at Scan: Group Comparison"
)

dataframe_to_word_table(
    doc,
    round_df(sex_ttest_results_df, 3),
    "Sex Distribution: Group Comparison"
)

dataframe_to_word_table(
    doc,
    round_df(mage_ttest_results_df, 3),
    "Maternal Age at Birth: Group Comparison"
)

dataframe_to_word_table(
    doc,
    round_df(group_pvals_df, 3),
    "Region-wise ANCOVA Results (Group Effect)"
)

doc.save("EA_CSF_DS_Results_Tables_rounded.docx")

In [19]:
from docx import Document

# Create document
doc = Document()
doc.add_heading("AAL Region Mapping for Regions Identified in ANCOVA Analysis", level=1)

# AAL mapping table data
aal_mapping = [
    (17, "Hippocampus", "Left"),
    (18, "Hippocampus", "Right"),
    (29, "Insula", "Left"),
    (30, "Insula", "Right"),
    (47, "Calcarine Cortex", "Left"),
    (56, "Lingual Gyrus", "Right"),
    (83, "Middle Temporal Gyrus", "Left"),
    (84, "Middle Temporal Gyrus", "Right"),
    (85, "Temporal Pole (Middle Temporal)", "Left"),
    (86, "Temporal Pole (Middle Temporal)", "Right"),
    (15, "Parahippocampal Gyrus", "Left"),
    (55, "Lingual Gyrus", "Left"),
    (81, "Superior Temporal Gyrus", "Left"),
    (11, "Rectus Gyrus", "Left"),
    (13, "Olfactory Cortex", "Left"),
    (16, "Parahippocampal Gyrus", "Right"),
    (46, "Calcarine Cortex", "Right"),
    (25, "Middle Occipital Gyrus", "Left"),
    (90, "Inferior Temporal Gyrus", "Right"),
    (31, "Anterior Cingulate Cortex", "Left"),
    (52, "Cuneus", "Right"),
    (23, "Middle Frontal Gyrus", "Left"),
    (4,  "Precentral Gyrus", "Right"),
    (82, "Superior Temporal Gyrus", "Right"),
    (14, "Olfactory Cortex", "Right"),
]

# Create table
table = doc.add_table(rows=1, cols=3)
table.style = "Table Grid"

# Header
hdr_cells = table.rows[0].cells
hdr_cells[0].text = "AAL Region Number"
hdr_cells[1].text = "Anatomical Region"
hdr_cells[2].text = "Hemisphere"

# Populate table
for region_num, region_name, hemisphere in aal_mapping:
    row_cells = table.add_row().cells
    row_cells[0].text = str(region_num)
    row_cells[1].text = region_name
    row_cells[2].text = hemisphere

# Save document
doc.save("AAL_Region_Mapping_Table.docx")

In [21]:
[c for c in df.columns if "norm" in str(c).lower()]

[]

In [25]:
# Look for likely covariate column names in your dataframe
sex_like  = [c for c in df.columns if "sex" in str(c).lower() or "gender" in str(c).lower()]
mage_like = [c for c in df.columns if "maternal" in str(c).lower() and "age" in str(c).lower()]
age_like  = [c for c in df.columns if ("age" in str(c).lower() and ("scan" in str(c).lower() or "mri" in str(c).lower()))
             or "age_at" in str(c).lower()]

print("Sex-like columns:", sex_like)
print("Maternal-age-like columns:", mage_like)
print("Age-at-scan-like columns:", age_like)

Sex-like columns: ['V06 demographics,Sex', 'V12-CVD demographics,Sex', 'V06-CVD demographics,Sex', 'V24-CVD demographics,Sex', 'V24 demographics,Sex', 'V12 demographics,Sex', 'V24-Sleep demographics,Sex']
Maternal-age-like columns: ['V06 demographics,Maternal_age_at_birth', 'V12-CVD demographics,Maternal_age_at_birth', 'V06-CVD demographics,Maternal_age_at_birth', 'V24-CVD demographics,Maternal_age_at_birth', 'V24 demographics,Maternal_age_at_birth', 'V12 demographics,Maternal_age_at_birth', 'V24-Sleep demographics,Maternal_age_at_birth']
Age-at-scan-like columns: ['V06 demographics,age_at_MRI', 'V12-CVD demographics,age_at_MRI', 'V06-CVD demographics,age_at_MRI', 'V24-CVD demographics,age_at_MRI', 'V24 demographics,age_at_MRI', 'V12 demographics,age_at_MRI', 'V24-Sleep demographics,age_at_MRI', 'V06 demographics,Maternal_age_at_birth', 'V12-CVD demographics,Maternal_age_at_birth', 'V06-CVD demographics,Maternal_age_at_birth', 'V24-CVD demographics,Maternal_age_at_birth', 'V24 demograp

In [ ]:
import numpy as np
import pandas as pd
import re

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import (
    roc_auc_score, accuracy_score, balanced_accuracy_score,
    confusion_matrix, classification_report
)

# ---------------------------
# 0) Choose your working dataframe (IMPORTANT)
# ---------------------------
# Use the same normalized tables you used for ANCOVA (these should contain *_norm_ICV columns)
df = pd.concat([
    ds_infant_norm.assign(Group="DS"),
    ds_control_norm.assign(Group="Control")
], ignore_index=True)

# ---------------------------
# 1) Detect AAL feature columns (your project uses: "17_norm_ICV", etc.)
# ---------------------------
aal_cols = [c for c in df.columns if str(c).strip().lower().endswith("_norm_icv")]
print("Found", len(aal_cols), "AAL feature columns")
print("Example AAL columns:", aal_cols[:10])


Found 78 AAL feature columns
Example AAL columns: ['1_norm_ICV', '3_norm_ICV', '5_norm_ICV', '7_norm_ICV', '9_norm_ICV', '11_norm_ICV', '13_norm_ICV', '15_norm_ICV', '17_norm_ICV', '19_norm_ICV']

Covariate missingness:
Sex_cov            0.352113
MaternalAge_cov    0.366197
AgeAtScan_cov      1.000000
Name: fraction_missing, dtype: float64
After covariate+label filtering: 0
After covariate+label filtering: 0

Using n subjects: 0 | n AAL features: 78


/var/folders/6k/mcmp6y590hd7ghs69ccp6y5w0000gn/T/ipykernel_47891/1267746231.py:79: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  visit_dt = pd.to_datetime(visit_raw, errors="coerce", infer_datetime_format=True)
/var/folders/6k/mcmp6y590hd7ghs69ccp6y5w0000gn/T/ipykernel_47891/1267746231.py:80: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dob_dt   = pd.to_datetime(dob_raw,   errors="coerce", infer_datetime_format=True)


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [35]:
import numpy as np
import pandas as pd
import re

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import (
    roc_auc_score, accuracy_score, balanced_accuracy_score,
    confusion_matrix, classification_report
)

# ============================================================
# 0) Build modeling dataframe (matches your ANCOVA approach)
# ============================================================
# Assumes ds_infant_norm and ds_control_norm already exist in your notebook.
df = pd.concat(
    [ds_infant_norm.assign(Group="DS"),
     ds_control_norm.assign(Group="Control")],
    ignore_index=True
).copy()

# ============================================================
# 1) Helper: coalesce two possible columns (CVD vs non-CVD)
# ============================================================
def coalesce_cols(df, col1, col2):
    s1 = df[col1] if col1 in df.columns else pd.Series(np.nan, index=df.index)
    s2 = df[col2] if col2 in df.columns else pd.Series(np.nan, index=df.index)
    return s1.combine_first(s2)

# ============================================================
# 2) Covariates (use V06 columns, coalescing CVD/non-CVD)
#    These names match your merged2.csv schema.
# ============================================================
sex_col_cvd    = "V06-CVD demographics,Sex"
sex_col_noncvd = "V06 demographics,Sex"

mage_col_cvd    = "V06-CVD demographics,Maternal_age_at_birth"
mage_col_noncvd = "V06 demographics,Maternal_age_at_birth"

visit_date_col_cvd    = "V06-CVD demographics,Appointment_MRI_date"
visit_date_col_noncvd = "V06 demographics,Appointment_MRI_date"

dob_col_cvd    = "V06-CVD demographics,DoB"
dob_col_noncvd = "V06 demographics,DoB"

# ---- Sex (Female=0, Male=1) ----
sex_raw = coalesce_cols(df, sex_col_cvd, sex_col_noncvd).astype(str).str.strip().str.lower()
sex_map = {"female": 0, "f": 0, "0": 0, "male": 1, "m": 1, "1": 1}
df["Sex_cov"] = sex_raw.map(sex_map)

# ---- Maternal age ----
mage_raw = coalesce_cols(df, mage_col_cvd, mage_col_noncvd).replace({".": np.nan, "": np.nan})
df["MaternalAge_cov"] = pd.to_numeric(mage_raw, errors="coerce")

# ---- Age at scan (years) = VisitDate - DoB ----
visit_raw = coalesce_cols(df, visit_date_col_cvd, visit_date_col_noncvd).replace({".": np.nan, "": np.nan})
dob_raw   = coalesce_cols(df, dob_col_cvd, dob_col_noncvd).replace({".": np.nan, "": np.nan})

visit_dt = pd.to_datetime(visit_raw, errors="coerce")
dob_dt   = pd.to_datetime(dob_raw, errors="coerce")
df["AgeAtScan_cov"] = (visit_dt - dob_dt).dt.days / 365.25

cov_cols = ["Sex_cov", "MaternalAge_cov", "AgeAtScan_cov"]

print("Covariate non-missing counts:")
for c in cov_cols:
    print(f"  {c}: {df[c].notna().sum()} / {len(df)}")

# ============================================================
# 3) Labels
# ============================================================
if "Group" not in df.columns:
    raise ValueError("Expected a 'Group' column after concatenation, but it is missing.")

y = df["Group"].astype(str).str.strip().str.lower().map({"control": 0, "ds": 1})
if y.isna().any():
    bad = sorted(df.loc[y.isna(), "Group"].astype(str).unique().tolist())
    raise ValueError(f"Unrecognized Group labels: {bad}. Expected only 'DS' and 'Control'.")
y = y.astype(int).values

# ============================================================
# 4) Detect AAL features
#    Case A: already have *_norm_ICV columns
#    Case B: have raw numeric AAL columns -> create *_norm_ICV using ICV
# ============================================================
norm_icv_cols = [c for c in df.columns if str(c).strip().lower().endswith("_norm_icv")]

if len(norm_icv_cols) > 0:
    aal_cols = norm_icv_cols
    print(f"Using {len(aal_cols)} existing *_norm_ICV AAL features.")
else:
    # raw AAL columns are numeric strings like "0", "1", "3", ... (exclude "0.1" etc)
    raw_aal_cols = [c for c in df.columns if re.fullmatch(r"\d+", str(c).strip())]
    if len(raw_aal_cols) == 0:
        raise ValueError(
            "No *_norm_ICV columns and no raw numeric AAL columns found. "
            "Check which dataframe you're using."
        )

    # ICV column (from your merged2.csv schema)
    icv_col = "vol05_mm^3_ICV"
    if icv_col not in df.columns:
        # fallback: try any column containing 'icv'
        icv_candidates = [c for c in df.columns if "icv" in str(c).lower()]
        raise ValueError(f"ICV column '{icv_col}' not found. Candidates: {icv_candidates}")

    df[icv_col] = pd.to_numeric(df[icv_col], errors="coerce")

    # create normalized columns
    aal_cols = []
    for c in raw_aal_cols:
        newc = f"{c}_norm_ICV"
        df[newc] = pd.to_numeric(df[c], errors="coerce") / df[icv_col]
        aal_cols.append(newc)

    print(f"Created {len(aal_cols)} *_norm_ICV features from raw AAL columns using {icv_col}.")

print("Example AAL columns:", aal_cols[:10])

# ============================================================
# 5) Build model dataframe
#    Key: require covariates + label, then median-impute AAL features
# ============================================================
df_model = df[aal_cols + cov_cols].copy()
df_model["y"] = y

# only require covariates + y (do NOT require all AAL cols present)
df_model = df_model.dropna(subset=cov_cols + ["y"])
print("After requiring covariates + label:", len(df_model), "subjects")

# numeric conversion + median impute AAL
for c in aal_cols:
    df_model[c] = pd.to_numeric(df_model[c], errors="coerce")
df_model[aal_cols] = df_model[aal_cols].apply(lambda s: s.fillna(s.median()), axis=0)

X_df = df_model[aal_cols + cov_cols].copy()
y = df_model["y"].astype(int).values

if len(df_model) < 10:
    raise ValueError("Too few subjects remain after covariate filtering. Check date parsing / column names.")

print("Final n subjects:", len(df_model), "| n features:", len(aal_cols))

# ============================================================
# 6) Leakage-safe residualizer (fit within each CV fold)
# ============================================================
class ResidualizeFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, feature_cols, covariate_cols):
        self.feature_cols = feature_cols
        self.covariate_cols = covariate_cols

    def fit(self, X, y=None):
        F = X[self.feature_cols].to_numpy(dtype=float)
        C = X[self.covariate_cols].to_numpy(dtype=float)
        C_design = np.column_stack([np.ones((C.shape[0], 1)), C])  # intercept
        self.betas_, *_ = np.linalg.lstsq(C_design, F, rcond=None)
        return self

    def transform(self, X):
        F = X[self.feature_cols].to_numpy(dtype=float)
        C = X[self.covariate_cols].to_numpy(dtype=float)
        C_design = np.column_stack([np.ones((C.shape[0], 1)), C])
        return F - (C_design @ self.betas_)

# ============================================================
# 7) Pipeline + CV evaluation
# ============================================================
pipe = Pipeline([
    ("resid", ResidualizeFeatures(feature_cols=aal_cols, covariate_cols=cov_cols)),
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(C=1.0, class_weight="balanced", max_iter=20000))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_predict(pipe, X_df, y, cv=cv, method="decision_function")
preds = (scores > 0).astype(int)

auc  = roc_auc_score(y, scores)
acc  = accuracy_score(y, preds)
bacc = balanced_accuracy_score(y, preds)
cm   = confusion_matrix(y, preds)

print("\n--- Cross-validated performance (5-fold) ---")
print(f"ROC AUC:           {auc:.3f}")
print(f"Accuracy:          {acc:.3f}")
print(f"Balanced Accuracy: {bacc:.3f}")
print("\nConfusion matrix (rows=true, cols=pred):\n", cm)
print("\nClassification report:\n", classification_report(y, preds, target_names=["Control (0)", "DS (1)"]))

# ============================================================
# 8) Fit final model for weights
# ============================================================
pipe.fit(X_df, y)
weights = pipe.named_steps["svm"].coef_.ravel()

svm_weights_df = pd.DataFrame({
    "feature": aal_cols,
    "weight": weights,
    "abs_weight": np.abs(weights)
}).sort_values("abs_weight", ascending=False).reset_index(drop=True)

print("\nTop 15 contributing AAL features by |weight|:")
display(svm_weights_df.head(15))

Covariate non-missing counts:
  Sex_cov: 57 / 71
  MaternalAge_cov: 56 / 71
  AgeAtScan_cov: 71 / 71
Using 78 existing *_norm_ICV AAL features.
Example AAL columns: ['1_norm_ICV', '3_norm_ICV', '5_norm_ICV', '7_norm_ICV', '9_norm_ICV', '11_norm_ICV', '13_norm_ICV', '15_norm_ICV', '17_norm_ICV', '19_norm_ICV']
After requiring covariates + label: 56 subjects
Final n subjects: 56 | n features: 78

--- Cross-validated performance (5-fold) ---
ROC AUC:           0.892
Accuracy:          0.821
Balanced Accuracy: 0.823

Confusion matrix (rows=true, cols=pred):
 [[26  6]
 [ 4 20]]

Classification report:
               precision    recall  f1-score   support

 Control (0)       0.87      0.81      0.84        32
      DS (1)       0.77      0.83      0.80        24

    accuracy                           0.82        56
   macro avg       0.82      0.82      0.82        56
weighted avg       0.82      0.82      0.82        56


Top 15 contributing AAL features by |weight|:


,feature,weight,abs_weight
0,44_norm_ICV,0.414208,0.414208
1,47_norm_ICV,0.376882,0.376882
2,70_norm_ICV,0.352117,0.352117
3,86_norm_ICV,0.346688,0.346688
4,25_norm_ICV,-0.329559,0.329559
5,58_norm_ICV,0.297400,0.297400
6,66_norm_ICV,-0.295685,0.295685
7,62_norm_ICV,0.281867,0.281867
8,63_norm_ICV,-0.270102,0.270102
9,23_norm_ICV,-0.259164,0.259164
